In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain_ollama import ChatOllama

In [3]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
import PyPDF2

In [4]:
llm = ChatOllama(model="mistral")

In [5]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [6]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [7]:
quiz_generation_prompt = ChatPromptTemplate.from_messages([
   HumanMessagePromptTemplate.from_template(TEMPLATE)
    ])

In [8]:
print(quiz_generation_prompt)

input_variables=['number', 'response_json', 'subject', 'text', 'tone'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['number', 'response_json', 'subject', 'text', 'tone'], input_types={}, partial_variables={}, template='\nText:{text}\nYou are an expert MCQ maker. Given the above text, it is your job to create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. \nMake sure the questions are not repeated and check all the questions to be conforming the text as well.\nMake sure to format your response like  RESPONSE_JSON below  and use it as a guide. Ensure to make {number} MCQs\n### RESPONSE_JSON\n{response_json}\n\n'), additional_kwargs={})]


In [9]:
quiz_chain = quiz_generation_prompt | llm

In [10]:
quiz_chain

ChatPromptTemplate(input_variables=['number', 'response_json', 'subject', 'text', 'tone'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['number', 'response_json', 'subject', 'text', 'tone'], input_types={}, partial_variables={}, template='\nText:{text}\nYou are an expert MCQ maker. Given the above text, it is your job to create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. \nMake sure the questions are not repeated and check all the questions to be conforming the text as well.\nMake sure to format your response like  RESPONSE_JSON below  and use it as a guide. Ensure to make {number} MCQs\n### RESPONSE_JSON\n{response_json}\n\n'), additional_kwargs={})])
| ChatOllama(model='mistral')

In [11]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [12]:
quiz_evaluation_prompt= ChatPromptTemplate.from_messages([
    HumanMessagePromptTemplate.from_template(TEMPLATE2)
])

In [13]:
review_chain = quiz_evaluation_prompt | llm

In [16]:
file_path =r"E:\mcqgen\data.txt"

In [17]:
with open(file_path,'r') as file:
    TEXT=file.read()

In [18]:
TEXT

'A black hole is a massive, compact astronomical object so dense that its gravity prevents anything from escaping, even light. Albert Einstein\'s theory of general relativity predicts that a sufficiently compact mass will form a black hole.[4] The boundary of no escape is called the event horizon. A black hole has a great effect on the fate and circumstances of an object crossing it, but has no locally detectable features according to general relativity.[5] In many ways, a black hole acts like an ideal black body, as it reflects no light.[6][7] Quantum field theory in curved spacetime predicts that event horizons emit Hawking radiation, with the same spectrum as a black body of a temperature inversely proportional to its mass. This temperature is of the order of billionths of a kelvin for stellar black holes, making it essentially impossible to observe directly.\n\nObjects whose gravitational fields are too strong for light to escape were first considered in the 18th century by John Mi

In [19]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [20]:
NUMBER=5 
SUBJECT="Space-Science"
TONE="simple"

In [23]:
quiz=quiz_chain.invoke({"number": NUMBER, "subject": SUBJECT, "tone": TONE, "text": TEXT, "response_json": json.dumps(RESPONSE_JSON)})

In [24]:
review=review_chain.invoke({"quiz": quiz.content, "subject": SUBJECT})

In [27]:
print(review.content)

 The quiz is appropriate for intermediate to advanced level Space-Science students. However, it might be challenging for beginners due to the technical vocabulary used in some questions. For example, question 4 could be rephrased as "Can a black hole be detected locally according to general relativity?"

Here's an updated version of the quiz with simplified language:

```
{
  "1": {
    "mcq": "What is a black hole?",
    "options": {
      "a": "A place where gravity is so strong that nothing, not even light, can get out.",
      "b": "An area from which nothing can escape.",
      "c": "A theoretical concept in astronomy.",
      "d": "An object that reflects all light."
    },
    "correct": "a"
  },
  "2": {
    "mcq": "Who first found a solution to Einstein's theory of general relativity that describes black holes?",
    "options": {
      "a": "John Michell",
      "b": "Pierre-Simon Laplace",
      "c": "Karl Schwarzschild",
      "d": "David Finkelstein"
    },
    "correct": "

In [29]:
output_text = review.content

In [30]:
quiz_json_str = output_text.split("```")[1].strip()


In [31]:
quiz_dict = json.loads(quiz_json_str)

In [32]:
quiz_dict

{'1': {'mcq': 'What is a black hole?',
  'options': {'a': 'A place where gravity is so strong that nothing, not even light, can get out.',
   'b': 'An area from which nothing can escape.',
   'c': 'A theoretical concept in astronomy.',
   'd': 'An object that reflects all light.'},
  'correct': 'a'},
 '2': {'mcq': "Who first found a solution to Einstein's theory of general relativity that describes black holes?",
  'options': {'a': 'John Michell',
   'b': 'Pierre-Simon Laplace',
   'c': 'Karl Schwarzschild',
   'd': 'David Finkelstein'},
  'correct': 'c'},
 '3': {'mcq': 'What is the name for the boundary where nothing can escape from a black hole?',
  'options': {'a': 'Event horizon',
   'b': 'Accretion disk',
   'c': 'Hawking radiation',
   'd': 'Quasar'},
  'correct': 'a'},
 '4': {'mcq': 'Can a black hole be seen locally according to general relativity?',
  'options': {'a': 'Yes, it can be seen.',
   'b': 'No, it cannot be seen.',
   'c': 'It may emit radiation.',
   'd': 'It is hard

In [35]:
quiz_table_data=[]
for key,value in quiz_dict.items():
    mcq=value['mcq']
    options= ",".join(
        [f"{option} : {option_value}"
         for option, option_value in value['options'].items()
         ]
    )
    correct=value['correct']
    quiz_table_data.append({
        "MCQ": mcq,
        "Options": options,
        "Correct": correct
    })

In [36]:
quiz_table_data

[{'MCQ': 'What is a black hole?',
  'Options': 'a : A place where gravity is so strong that nothing, not even light, can get out.,b : An area from which nothing can escape.,c : A theoretical concept in astronomy.,d : An object that reflects all light.',
  'Correct': 'a'},
 {'MCQ': "Who first found a solution to Einstein's theory of general relativity that describes black holes?",
  'Options': 'a : John Michell,b : Pierre-Simon Laplace,c : Karl Schwarzschild,d : David Finkelstein',
  'Correct': 'c'},
 {'MCQ': 'What is the name for the boundary where nothing can escape from a black hole?',
  'Options': 'a : Event horizon,b : Accretion disk,c : Hawking radiation,d : Quasar',
  'Correct': 'a'},
 {'MCQ': 'Can a black hole be seen locally according to general relativity?',
  'Options': 'a : Yes, it can be seen.,b : No, it cannot be seen.,c : It may emit radiation.,d : It is hard to observe directly.',
  'Correct': 'b'},
 {'MCQ': 'Where are supermassive black holes thought to exist?',
  'Opti

In [37]:
df=pd.DataFrame(quiz_table_data)

In [38]:
df

,MCQ,Options,Correct
0,What is a black hole?,a : A place where gravity is so strong that no...,a
1,Who first found a solution to Einstein's theor...,"a : John Michell,b : Pierre-Simon Laplace,c : ...",c
2,What is the name for the boundary where nothin...,"a : Event horizon,b : Accretion disk,c : Hawki...",a
3,Can a black hole be seen locally according to ...,"a : Yes, it can be seen.,b : No, it cannot be ...",b
4,Where are supermassive black holes thought to ...,"a : In the centers of galaxies.,b : At the edg...",a


In [39]:
df.to_csv("quiz.csv", index=False)